# Data Exploration

### Use cases for About Excel/SPSS/Google Sheets
* experimental psych - report
* planning for grad school - organize timelines & stuff

### Like 
* ease of calculations
* it's organized - generates organized summaries/reports
* functions built in that can do a lot of things
* can do calculation on entire row/multiple columns

### Dislike
* data input is manual/often cell by cell
* hard to navigate - confusing UI
* not great with large data files

### Things we'd like to do
*  

## Objectives
1. loading data from tables (excel files, csvs)

1. cleaning misformed data and missing values

1. filtering data using keywords and logical constraints

1. computing summary statistics

1. aggregating data via pivot tables


# Load Data Using Pandas

Pandas is a Python library (set of functions somebody else wrote) for doing data analysis. 

https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf

In [1]:
import pandas as pd #import is how we load libraries
pd.__version__

'0.24.1'

In [ ]:
# ../data/abuse.csv is where I store the file - change to your location
# file is stored in df variable
df = pd.read_csv("../data/abuse.csv")

# Quick peek at the data:
* __head__: first 5 rows
* __tail__: last 5 rows

In [ ]:
df.head()

In [ ]:
df.tail()

# Snapshots of the spreadsheet
* info - column datatypes
* describe - statistics

In [ ]:
df.info()

In [ ]:
df.describe()

# Let's look at columns
* Remember dictionaries? Same thing:
    * df['column name']
    * df[['list of columns']]

In [ ]:
df.columns

In [ ]:
df['Male-%'].head()

Let's get characteristics too...

In [ ]:
df[['characteristic', 'Male-%']].head() #remember head is only top 5

# How is the data organized? 

![tidy data organization - rows = observations, columns=variables, measurments=cells](figs/L06/data_formatting.png)
Munzner and E. Maguire, Visualization analysis & design. Boca Raton, FL: CRC Press, 2015.

 ![image of axis, where rows=axis 0, columns = axis 1](figs/L06/axis.jpg)
[stackoverflow](https://stackoverflow.com/questions/25773245/ambiguity-in-pandas-dataframe-numpy-array-axis-definition)

# Let's see that column

In [ ]:
# matplotlib is one of the Python visualization libraries
import matplotlib
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
_ = df[['characteristic', 'Male-%']].plot.bar(ax=ax)
_ = ax.set_xticklabels(zip(df['characteristic'], df['race-ethnicity']))

# Practice:
* Select and plot a different column

# How do we select rows?

![diagram of boolean indexing. Shows first row of numbers, second row of true first values, rows in third column are blank where rows in second are false, remaining values in third form 4th column](figs/L06/masking.png)
Modified from [Software Carpentry](https://v4.software-carpentry.org/matrix/indexing.html)

In [ ]:
df.head()

Where characteristic = total is a double count, so let's remove that!

In [ ]:
df['characteristic'].str.match("Total").head()

In [ ]:
total_rows = df['characteristic'].str.match("Total")
df[total_rows]

Negation of a condition (not) is `~`

In [ ]:
df[~total_rows].head()


# Still double counting 'cause of total race ethnicity, let's use compound logic

In [ ]:
total_rows = (df['characteristic'].str.match("Total") | df['race-ethnicity'].str.match("Total"))
df[total_rows]

In [ ]:
dfc = df[~total_rows]
dfc.head()

In [ ]:
dfc.describe() # hos is this different from the one that includes total?

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
_ = dfc.plot.bar(ax=ax)
_ = ax.set_xticklabels(zip(dfc['characteristic'], dfc['race-ethnicity']))

That's a bit busy, let's just pull out 'White'

In [ ]:
white = dfc[dfc['race-ethnicity'].str.match("White")]
white

In [ ]:
fig, ax = plt.subplots()
_ = white.plot.bar(ax =ax)
_ = ax.set_xticklabels(white['characteristic'])

# Why are half the columns missing?
comparing apples to oranges to cats

In [ ]:
dfc.columns

In [ ]:
percent = dfc[['characteristic', 'race-ethnicity','Male-%', 'Female-%']]
percent

In [ ]:
# Lets look at this subset
fig, ax = plt.subplots()
_ = percent[percent['race-ethnicity'].str.match('White')].plot.bar(ax=ax)
_ = ax.set_xticklabels(percent['characteristic'])

# Practice:
1. Try a different race-ethnicity & a different set of columns
2. Try an age group

# Can we see characteristic by race?

In [ ]:
table = df.pivot(index='characteristic', columns='race-ethnicity', 
                    values=['Male-%', 'Female-%', 'Total-%'])


In [ ]:
table

In [ ]:
_ = table['Male-%'].plot.bar()

In [ ]:
# clean it up a bit using stacked bar plots
_ = table['Male-%'].plot.bar(stacked=True)

In [ ]:
# Switch the grouping by flipping the table
table['Male-%'].T

In [ ]:
_ = table['Male-%'].T.plot.bar(stacked=True)

# Practice:
1. See how women differ
2. Try a different column set

# Let's do some math!

In [ ]:
# Let's use columwise summation (axis=1) to confirm that the male and female estimates sum to the total
dfc[['Male-estimate', 'Female-estimate']].sum(axis=1)

In [ ]:
#let's see where it's off a bit
dfc[~(dfc[['Male-estimate', 'Female-estimate']].sum(axis=1) == dfc['Total-estimate'])]

In [ ]:
# mean per column:
dfc.mean()

In [ ]:
# full summary stats
dfc.describe()

In [ ]:
# Standard deviation
dfc.groupby(['race-ethnicity']).std()

In [ ]:
# What if we want the mean for each race-ethnicity?
estimates = dfc.groupby(['race-ethnicity'])[['Male-estimate', 'Female-estimate', 'Total-estimate']].sum()

In [ ]:
estimates

In [ ]:
# remove total so we're not double counting on the visualization
est_sex = estimates[['Male-estimate', 'Female-estimate']]
_ = est_sex.plot.bar(stacked=True)

In [ ]:
#pictures need lots of polishing, this is just exploratory
_ = est_sex.plot.pie(subplots=True, figsize=(10,5))

In [ ]:
_ = est_sex.T.plot.bar(stacked=True)

In [ ]:
# let's look at gender, which means removing totals
_ = est_sex.T['White'].plot.pie()

# Practice
Try plotting a different race-ethnicity

In [ ]:
# Let's use a boxplot to visualize the different grou
import seaborn as sns
sns.boxplot(x = 'race-ethnicity', y = 'Male-estimate', data=dfc)

# Practice 
Get the mean for each demographic/characteristic (ignore race)

# Practice Project:
1. Open the dependency file
2. Compare dependency & abuse rates (with figures): have summaries for each race, sex, & demographic

# How do we join two datasets?
![table merge where 1st row is scanned and on match with element in second row, new row is created with elements of both](figs/L06/merge.gif)

Source [Randy Au, Can we stop with the SQL JOINs venn diagrams insanity?](https://towardsdatascience.com/can-we-stop-with-the-sql-joins-venn-diagrams-insanity-16791d9250c3?sk=f8bfa36658362ee6d54951681967a45b)


In [ ]:
df2 = pd.read_csv("../data/dependency.csv")

In [ ]:
df2.head()

In [ ]:
# need both characteristic and race-ethnicty for row uniquiness
pd.merge(df, df2, on=['characteristic', 'race-ethnicity'])

In [ ]:
#lets use better identifiers than x, y
data = pd.merge(df, df2, on=['characteristic', 'race-ethnicity'], suffixes=('-abuse', '-dependency'))

In [ ]:
data.head()

In [ ]:
# let's get just estimate data
ecol = [est for est in data.columns if 'estimate' in est]
ecol

In [ ]:
estdf = data[['characteristic', 'race-ethnicity']+ecol]

In [ ]:
estdf.head()

# Practice
1. For each race/demo/sex, find if the abuse or dependency is higher, and the difference between the two
2. Visualize the difference